In [1]:
import sys
import torch
import numpy as np
import random
import os
import time
import skimage
from skimage import data
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imshow
import pdb
from models import create_Inet
import torch
import torchvision
import torchvision.transforms as transforms
from scipy import interpolate
from utils.image_folder import ImageFolder
from utils.utils import *
# from utils.Visualizer import Visualizer
import matplotlib.pyplot as plt
from IPython.display import clear_output
from options import DefaultConfig
opt = DefaultConfig()
torch.cuda.set_device(opt.gpu_ids)
torch.manual_seed(opt.seed)
torch.cuda.manual_seed_all(opt.seed)
np.random.seed(opt.seed)
random.seed(opt.seed)
torch.backends.cudnn.deterministic = True

Interaction Network: initialized


In [2]:
opt.dataroot_val = opt.dataroot + 'subval/'
dataset_val = ImageFolder(opt.dataroot_val, opt, 
                       transform=transforms.Compose([transforms.ToTensor()]))
dataset_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=opt.batch_size_val, shuffle=True)
dataset_val_size = len(dataset_val)
print('#training images = %d' % dataset_val_size)

#training images = 1000


In [3]:
model = create_Inet(opt)
path = 'I_auto_ep2_val_30.pkl'model.load_state_dict(torch.load(path, map_location='cuda:3').state_dict())
model.eval();
model.cuda();

Interaction net was created


In [4]:
import skimage
for i, data_raw in enumerate(dataset_loader_val):
    
    data_np = get_colorization_data(data_raw, opt, p=opt.sample_p, first_round=True)
    data = ToCuda(data_np)
    em_ab = model(data['gray'], data['clicks'], data['prev'])

    em_ab_np = em_ab.detach().cpu().numpy()
    psnr = PSNR(data_np, em_ab_np, opt, avg=True)
    break

/home/zhangy23/anaconda3/envs/vc/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [5]:
data_np['gray'][0,:,:,:].shape

(1, 224, 224)

In [6]:
em_img = np.append(data_np['gray'][1,:,:,:] * 100, em_ab_np[1,:,:,:] * 128, axis=0)
gt_img = np.append(data_np['gray'][1,:,:,:] * 100, data_np['ab'][1,:,:,:] * 128, axis=0)
em_img = np.transpose(em_img, (1, 2, 0))
gt_img = np.transpose(gt_img, (1, 2, 0))
em = skimage.color.lab2rgb(em_img)
gt = skimage.color.lab2rgb(gt_img)

/home/zhangy23/anaconda3/envs/vc/lib/python3.6/site-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 771 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
